### Inicialização

In [ ]:
# Pacotes

import pandas as pd
import numpy as np
import random

In [ ]:
# Leitura das planilhas

lineup = pd.read_excel('lineup_aplicacao.xlsx', sheet_name='Team')
status = pd.read_excel('status.xlsx', dtype='str')

### Definição das funções

In [ ]:
# Função para criar as colunas necessárias

def f_cria_variaveis(lineup):
    
    lineup['P_S']   = lineup['S']/lineup['PA']
    lineup['P_D']   = lineup['D']/lineup['PA']
    lineup['P_T']   = lineup['T']/lineup['PA']
    lineup['P_HR']  = lineup['HR']/lineup['PA']
    lineup['P_BB']  = lineup['BB']/lineup['PA']
    lineup['P_HBP'] = lineup['HBP']/lineup['PA']
    lineup['P_IBB'] = lineup['IBB']/lineup['PA']
    lineup['P_SO']  = lineup['SO']/lineup['PA']
    lineup['P_OUT'] = lineup['OUT']/lineup['PA']
    lineup['P_GDP'] = lineup['GDP']/lineup['PA']
    lineup['P_SH']  = lineup['SH']/lineup['PA']
    lineup['P_SF']  = lineup['SF']/lineup['PA']
    
    lineup = lineup[['INDEX_AUX', 'INDEX_FIXO', 'P_S', 'P_D', 'P_T', 'P_HR', 'P_BB', 'P_HBP', 'P_IBB', 'P_SO', 'P_OUT', 'P_GDP', 'P_SH', 'P_SF']].copy()
    
    return lineup

In [ ]:
# Função para simular uma partida

def f_simula_partida(lineup):
    
    # Inicializando o INDEX_FIXO
    lineup['INDEX_FIXO'] = range(1, 10)
    
    # Parametros iniciais
    pa_outcomes = ['S', 'D', 'T', 'HR', 'BB', 'HBP', 'IBB', 'SO', 'OUT', 'GDP', 'SH', 'SF']

    outs = 0
    base1 = 0
    base2 = 0
    base3 = 0
    home_plate = 0
    runs = 0
    player = 1
    inn = 1
    
    status_atual = str(outs) + str(base1) + str(base2) + str(base3) + str(home_plate)
    
    # Pega o parametro inicial e atualiza ele de acordo com o outcome da aparicao no bastao
    while inn <= 9:
  
        #print("inn: ", inn)
        #print("player: ", player)
        #print("status_atual: ", status_atual)
        
        prob_pa_outcomes = lineup[lineup['INDEX_FIXO'] == player][['P_S', 'P_D', 'P_T', 'P_HR', 'P_BB', 'P_HBP', 'P_IBB', 'P_SO', 'P_OUT', 'P_GDP', 'P_SH', 'P_SF']].copy()
        prob_pa_outcomes = prob_pa_outcomes.iloc[0].tolist().copy()
        pa_outcome = random.choices(pa_outcomes, prob_pa_outcomes)[0]
        #print("pa_outcome: ", pa_outcome)
        
        status_pos = status[status['STATUS'] == status_atual][pa_outcome].tolist()[0]
        #print("status_pos: ", status_pos)
        
        outs = int(status_pos[0])
        base1 = int(status_pos[1])
        base2 = int(status_pos[2])
        base3 = int(status_pos[3])
        home_plate = int(status_pos[4])
        
        runs = runs + int(home_plate)
        home_plate = 0
        if outs == 3:
            outs = 0; inn = inn + 1; base1 = 0; base2 = 0; base3 = 0
        player = player + 1
        if player == 10:
            player = 1
        
        #print("outs: ", outs)
        #print("runs: ", runs)
        #print("-------------------------------")
        
        status_atual = str(outs) + str(base1) + str(base2) + str(base3) + str(home_plate)
    
    return runs

In [ ]:
# Função para simular várias partidas e armazenar o resultado

def f_simula_temporada(lineup, games):

    runs_game = [-1] * games

    # Roda a simulacao de uma partida varias vezes
    for i in range(games):
        runs_game[i] = f_simula_partida(lineup)
    
    #print("Partidas: ", games)
    #print("Total de corridas marcadas: ", round(sum(runs_game), 2))
    #print("Media de corridas marcadas: ", round(np.mean(runs_game), 2))
    #print("Desvio padrão das corridas marcadas: ", round(np.std(runs_game), 2))
    #print("")
    
    return np.mean(runs_game)

In [ ]:
# Função para reordenar o lineup todo, aleatoriamente

def f_reordena_lineup_full(lineup_original, seed):
    
    lineup_reordenado = lineup_original.sample(frac=1, random_state=seed)
    return lineup_reordenado

In [ ]:
# Função para reordenar o lineup, mudando k linhas, aleatoriamente

def f_reordena_lineup_k(lineup_original, index_col, k):
    
    indices_para_trocar = random.sample(lineup_original.index.tolist(), k)
    indices_para_trocar_pos = indices_para_trocar[1:] + indices_para_trocar[:1]
    
    lineup_reordenado = lineup_original.copy()
    
    for i in range(k):
        lineup_reordenado.loc[indices_para_trocar[i]] = lineup_original.loc[indices_para_trocar_pos[i]]
    
    index_reordenado = lineup_reordenado[index_col]
    
    return index_reordenado, lineup_reordenado

In [ ]:
# Funcao que roda o otimizador k-opt swap

def f_otimiza_lineup(lineup, qtd_max_parada):

    qtd_parada = 1
    qtd_iteracoes = 1
    melhor_lineup = lineup.copy()
    runs_game_melhor_lineup = f_simula_temporada(melhor_lineup, 162)
    
    print("Média corridas lineup original: ", round(runs_game_melhor_lineup, 4))
    
    while qtd_parada <= qtd_max_parada:
        
        k = random.choice([2, 3])
        
        index_reordenado, lineup_proposto = f_reordena_lineup_k(melhor_lineup, 'INDEX_AUX', k)
        
        runs_game_lineup_proposto = f_simula_temporada(lineup_proposto, 162)
        print("\n")
        print("Iteração: ", qtd_iteracoes)
        print("Critério de parada: ", qtd_parada, " de ", qtd_max_parada)
        print("Média corridas lineup proposto: ", round(runs_game_lineup_proposto, 4))
        
        qtd_parada += 1
        qtd_iteracoes += 1
        
        if runs_game_lineup_proposto > runs_game_melhor_lineup:
            runs_game_melhor_lineup = runs_game_lineup_proposto
            melhor_lineup = lineup_proposto
            ordenacao_melhor_lineup = melhor_lineup['INDEX_AUX']
            qtd_parada = 1
            print('Achou uma solução melhor!')
            print(index_reordenado)
            
    return runs_game_melhor_lineup, ordenacao_melhor_lineup

### Aplicação do simulador

In [ ]:
# Cria colunas necessárias

lineup = f_cria_variaveis(lineup)

In [ ]:
# Simula uma temporada

random.seed(12345)

f_simula_temporada(lineup, 162)

In [ ]:
# Aplicação do otimizador

random.seed(12345)

runs_game_melhor_lineup, ordenacao_melhor_lineup = f_otimiza_lineup(lineup, 100)